In [1]:
import numpy as np
import pandas as pd
import QUANTAXIS as qa

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.0.41
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````##``````###```` 
  ##````````##```##````````##`````##`````

In [3]:
can = qa.QA_fetch_stock_day_adv('000004', start='2017-06-01', end='2018-05-30').to_qfq()

In [4]:
raw = can.data.loc[:, ['open', 'close', 'high', 'low']].values

In [5]:
dates = can.data.date

In [6]:
from pyecharts import Grid, Bar, Line, Kline, Overlap

kline = Kline()
kline.add("日K", dates.tolist(), raw, is_datazoom_show=True)

overlap = Overlap()
overlap.add(kline)
overlap

In [7]:
mrg = []
mrg_cnt = []
mrg_date = []
raw = raw
raw_date = dates.tolist()
raw_idx = 0
mrg_idx = 0

while True:
    if raw_idx >= len(raw):
        break
    if raw_idx == 0:
        mrg.append(raw[raw_idx])
        mrg_date.append(raw_date[raw_idx])
        mrg_cnt.append(1)
        raw_idx += 1
        continue
    if (mrg[mrg_idx][2] >= raw[raw_idx][2] and mrg[mrg_idx][3] <= raw[raw_idx][3]) or (mrg[mrg_idx][2] <= raw[raw_idx][2] and mrg[mrg_idx][3] >= raw[raw_idx][3]):
        if mrg_idx == 0 or mrg[mrg_idx-1][2] <= mrg[mrg_idx][2]:
            mrg[mrg_idx][0] = max(mrg[mrg_idx][0], raw[raw_idx][0])
            mrg[mrg_idx][1] = max(mrg[mrg_idx][1], raw[raw_idx][1])
            mrg[mrg_idx][2] = max(mrg[mrg_idx][2], raw[raw_idx][2])
            mrg[mrg_idx][3] = max(mrg[mrg_idx][3], raw[raw_idx][3])
        else:
            mrg[mrg_idx][0] = min(mrg[mrg_idx][0], raw[raw_idx][0])
            mrg[mrg_idx][1] = min(mrg[mrg_idx][1], raw[raw_idx][1])
            mrg[mrg_idx][2] = min(mrg[mrg_idx][2], raw[raw_idx][2])
            mrg[mrg_idx][3] = min(mrg[mrg_idx][3], raw[raw_idx][3])
        mrg_date[mrg_idx] = raw_date[raw_idx]
        mrg_cnt[mrg_idx] += 1
        raw_idx += 1
        continue
    mrg.append(raw[raw_idx])
    mrg_date.append(raw_date[raw_idx])
    mrg_cnt.append(1)
    raw_idx += 1
    mrg_idx += 1

[(mrg_date[idx], mrg[idx][2], mrg[idx][3], mrg_cnt[idx]) for idx in range(len(mrg))]

[(Timestamp('2017-06-15 00:00:00'), 32.400000000000006, 32.400000000000006, 1),
 (Timestamp('2017-06-16 00:00:00'), 29.160000000000007, 29.160000000000007, 1),
 (Timestamp('2017-06-19 00:00:00'), 28.78, 26.24, 1),
 (Timestamp('2017-06-22 00:00:00'), 26.59, 26.01, 3),
 (Timestamp('2017-06-23 00:00:00'), 25.96, 24.8, 1),
 (Timestamp('2017-06-26 00:00:00'), 26.15, 25.12, 1),
 (Timestamp('2017-06-27 00:00:00'), 26.659999999999993, 25.919999999999995, 1),
 (Timestamp('2017-07-04 00:00:00'), 25.759999999999994, 25.259999999999994, 5),
 (Timestamp('2017-07-05 00:00:00'), 25.479999999999993, 25.179999999999993, 1),
 (Timestamp('2017-07-06 00:00:00'), 25.39999999999999, 25.159999999999993, 1),
 (Timestamp('2017-07-07 00:00:00'), 25.359999999999992, 25.07999999999999, 1),
 (Timestamp('2017-07-10 00:00:00'), 25.339999999999993, 24.889999999999993, 1),
 (Timestamp('2017-07-11 00:00:00'), 25.099999999999994, 24.64999999999999, 1),
 (Timestamp('2017-07-14 00:00:00'), 24.389999999999993, 23.699999999

In [8]:
kline = Kline()
kline.add("日K", mrg_date, mrg, is_datazoom_show=True, 
          mark_point=[{'coord':[mrg_date[-10], mrg[-10][2] * 1.03], 'name':'test'}], 
          mark_point_symbol='circle', mark_point_symbolsize=10)
kline
#overlap = Overlap()
#overlap.add(kline)
#overlap

In [28]:
def higher(x, y):
    return x[2] >= y[2] and x[3] >= y[3]

def lower(x, y):
    return x[2] <= y[2] and x[3] <= y[3]

end = []
last = 0
for idx, cell in enumerate(mrg):
    insert = False
    if idx < 2:
        continue
    if (higher(mrg[idx-1], mrg[idx-2]) and higher(mrg[idx-1], cell)):
        end.append([idx-1, 1, True])
        insert = True
    elif (lower(mrg[idx-1], mrg[idx-2]) and lower(mrg[idx-1], cell)):
        end.append([idx-1, 0, True])
        insert = True
    
    if insert and len(end)>1 and end[-1][1] == end[-2][1]:
        if ((end[-1][1] == 1 and mrg[end[-1][0]][2] > mrg[end[-2][0]][2]) or 
            (end[-1][1] == 0 and mrg[end[-1][0]][3] < mrg[end[-2][0]][3])):
            if not(idx-2 == last and mrg_cnt[last] + mrg_cnt[last+1] > 2 or idx-3 > last) and len(end) > 2:
                end.pop(-2)
            else:
                end[-2][2] = False
        else:
            end.pop()
            insert = False
    elif insert and not(idx-2 == last and mrg_cnt[last] + mrg_cnt[last+1] > 2 or idx-3 > last):
        end.pop()
        insert = False
    
    if insert:
        last = idx

[(item[0], str(mrg_date[item[0]]), 'top' if item[1] else 'bottom', 'valid' if item[2] else 'not') for item in end]

[(4, '2017-06-23 00:00:00', 'bottom', 'not'),
 (15, '2017-07-19 00:00:00', 'bottom', 'valid'),
 (23, '2017-08-09 00:00:00', 'top', 'not'),
 (27, '2017-08-22 00:00:00', 'top', 'valid'),
 (33, '2017-08-30 00:00:00', 'bottom', 'valid'),
 (39, '2017-09-12 00:00:00', 'top', 'not'),
 (41, '2017-09-15 00:00:00', 'top', 'valid'),
 (47, '2017-09-29 00:00:00', 'bottom', 'valid'),
 (52, '2017-10-18 00:00:00', 'top', 'valid'),
 (61, '2017-11-07 00:00:00', 'bottom', 'valid'),
 (69, '2017-11-22 00:00:00', 'top', 'valid'),
 (76, '2017-12-07 00:00:00', 'bottom', 'valid'),
 (81, '2017-12-15 00:00:00', 'top', 'valid'),
 (85, '2017-12-21 00:00:00', 'bottom', 'valid'),
 (90, '2018-01-04 00:00:00', 'top', 'valid'),
 (99, '2018-01-22 00:00:00', 'bottom', 'valid'),
 (101, '2018-01-25 00:00:00', 'top', 'valid'),
 (106, '2018-02-02 00:00:00', 'bottom', 'not'),
 (108, '2018-02-07 00:00:00', 'bottom', 'valid'),
 (113, '2018-02-27 00:00:00', 'top', 'not'),
 (117, '2018-03-07 00:00:00', 'top', 'not'),
 (119, '2018

In [10]:
valid = [x for x in end if x[2]]
valid

[[15, 0, True],
 [27, 1, True],
 [33, 0, True],
 [41, 1, True],
 [47, 0, True],
 [52, 1, True],
 [61, 0, True],
 [69, 1, True],
 [76, 0, True],
 [81, 1, True],
 [85, 0, True],
 [90, 1, True],
 [99, 0, True],
 [101, 1, True],
 [108, 0, True],
 [119, 1, True],
 [125, 0, True],
 [127, 1, True],
 [139, 0, True],
 [141, 1, True],
 [143, 0, True],
 [148, 1, True]]

In [29]:

class Line(object):
    def __init__(self, start_idx, end_idx, ochl):
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.is_bull = ochl[start_idx][2] < ochl[end_idx][2] and ochl[start_idx][3] < ochl[end_idx][3]
        self.start = ochl[start_idx][3] if self.is_bull else ochl[start_idx][2]
        self.end = ochl[end_idx][2] if self.is_bull else ochl[end_idx][3]
        self.high = self.end if self.is_bull else self.start
        self.low = self.start if self.is_bull else self.end
        span = self.high - self.low
        self.low_energy = self.low + span * 0.382 if self.is_bull else self.high - span * 0.382
        self.mid_energy = self.low + span * 0.5
        self.high_energy = self.low + span * 0.382 if not self.is_bull else self.high - span * 0.382
        self.first_target = self.high + span * 0.618 if self.is_bull else self.low - span * 0.618
        self.second_target = self.high + span * 1.618 if self.is_bull else self.low - span * 1.618
        self.valid = True
    
    def tostring(self):
        return '{} {} to {}'.format(self.start_idx, 'up' if self.is_bull else 'down', self.end_idx)


In [13]:
lines = [Line(a[0], b[0], mrg, mrg_date) for a, b in zip(valid[:-1], valid[1:])]

In [33]:
def find_lines(points):

    lines = [Line(points[0][0], points[1][0], mrg)]

    for idx, point in enumerate(points):
        if idx < 2:
            continue
        if point[1] == 0:
            lines = [l for l in lines if (mrg[point[0]][1] > l.low_energy and l.is_bull == True) or (mrg[point[0]][3] > l.second_target and l.is_bull == False)]
            starts = [l.start_idx for l in lines if l.is_bull == False and l.end > mrg[point[0]][3]]
            lines.extend([Line(x, point[0], mrg) for x in starts] + [Line(points[idx-1][0], point[0], mrg)])
        else:
            lines = [l for l in lines if (mrg[point[0]][1] < l.low_energy and l.is_bull == False) or (mrg[point[0]][2] < l.second_target and l.is_bull == True)]
            starts = [l.start_idx for l in lines if l.is_bull == True and l.end < mrg[point[0]][2]]
            lines.extend([Line(x, point[0], mrg) for x in starts] + [Line(points[idx-1][0], point[0], mrg)])

    return lines


In [41]:
[x.tostring() for x in find_lines(valid)]

['69 down to 76',
 '69 down to 85',
 '69 down to 99',
 '69 down to 108',
 '69 down to 108',
 '69 down to 108',
 '69 down to 125',
 '69 down to 125',
 '69 down to 125',
 '69 down to 139',
 '69 down to 139',
 '69 down to 139',
 '69 down to 139',
 '69 down to 139',
 '69 down to 139',
 '139 up to 141',
 '69 down to 143',
 '69 down to 143',
 '69 down to 143',
 '139 up to 148',
 '143 up to 148']

In [42]:
lines = find_lines(valid)
support = set()
pressure = set()

for l in lines:
    if l.is_bull:
        support.add((l.low_energy, '0.382 of {} up to {}'.format(l.start_idx, l.end_idx)))
        support.add((l.mid_energy, '0.5 of {} up to {}'.format(l.start_idx, l.end_idx)))
        support.add((l.high_energy, '0.618 of {} up to {}'.format(l.start_idx, l.end_idx)))
        pressure.add((l.high, 'high of {} up to {}'.format(l.start_idx, l.end_idx)))
        pressure.add((l.first_target, '1.618 of {} up to {}'.format(l.start_idx, l.end_idx)))
        pressure.add((l.second_target, '2.618 of {} up to {}'.format(l.start_idx, l.end_idx)))
    else:
        pressure.add((l.low_energy, '0.382 of {} down to {}'.format(l.start_idx, l.end_idx)))
        pressure.add((l.mid_energy, '0.5 of {} down to {}'.format(l.start_idx, l.end_idx)))
        pressure.add((l.high_energy, '0.618 of {} down to {}'.format(l.start_idx, l.end_idx)))
        support.add((l.low, 'low of {} down to {}'.format(l.start_idx, l.end_idx)))
        support.add((l.first_target, '1.618 of {} down to {}'.format(l.start_idx, l.end_idx)))
        support.add((l.second_target, '2.618 of {} down to {}'.format(l.start_idx, l.end_idx)))

sup = sorted(list(support), key=lambda x: x[0], reverse=True)
pre = sorted(list(pressure), key=lambda x: x[0])

print(sup)
print(pre)


[(22.56354, '0.618 of 143 up to 148'), (22.265, '0.5 of 143 up to 148'), (22.055480000000003, '0.618 of 139 up to 148'), (21.96646, '0.382 of 143 up to 148'), (21.7712, '0.618 of 139 up to 141'), (21.6, '0.5 of 139 up to 148'), (21.42, 'low of 69 down to 76'), (21.37, '0.5 of 139 up to 141'), (21.14452, '0.382 of 139 up to 148'), (21.05, 'low of 69 down to 99'), (21.02, 'low of 69 down to 85'), (21.0, 'low of 69 down to 143'), (20.9688, '0.382 of 139 up to 141'), (20.48, 'low of 69 down to 125'), (19.67, 'low of 69 down to 139'), (18.7, 'low of 69 down to 108'), (15.227640000000005, '1.618 of 69 down to 76'), (14.628980000000002, '1.618 of 69 down to 99'), (14.580440000000001, '1.618 of 69 down to 85'), (14.548080000000002, '1.618 of 69 down to 143'), (13.706720000000002, '1.618 of 69 down to 125'), (12.396140000000004, '1.618 of 69 down to 139'), (10.82668, '1.618 of 69 down to 108'), (5.2076400000000085, '2.618 of 69 down to 76'), (4.238980000000005, '2.618 of 69 down to 99'), (4.160